<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       NaiveBayesTextClassifierTrainer and NaiveBayesTextClassifierPredict function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>The NaiveBayesTextClassifierTrainer function calculates the conditional probabilities for token-category pairs, the prior probabilities, and the missing token probabilities for all categories. The trainer function trains the model with the probability values, and the predict function uses the values to classify documents into categories.<br>In this notebook we will see how we can use the NaiveBayesTextClassifierTrainer and Predict function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_NaiveBayesTextClassifierTrainerandPredict_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("textparser", ["complaints", "stop_words"])
load_example_data("NaiveBayesTextClassifierPredict",["complaints_tokens_test"])

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
complaints = DataFrame.from_table("complaints")
stop_words = DataFrame.from_table("stop_words")
complaints

<p style = 'font-size:16px;font-family:Arial'>As a first step we will use the TextParser function and tokenize our text. </p>

In [ ]:
# Tokenize the "text_column" and accumulate result by "doc_id" and "category".
complaints_tokenized = TextParser(data=complaints,
                                  text_column="text_data",
                                  object=stop_words,
                                  remove_stopwords=True,
                                  accumulate=["doc_id", "category"])

<p style = 'font-size:16px;font-family:Arial'>Now we will use NaiveBayesTextClassifierTrainer function to train our model.<br>
Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(NaiveBayesTextClassifierTrainer)

In [ ]:
# Example 1 : Calculate the conditional probabilities for token-category pairs.
NaiveBayesTextClassifierTrainer_out = NaiveBayesTextClassifierTrainer(data=complaints_tokenized.result,
                                                                      token_column="token",
                                                                      doc_category_column="category")

In [ ]:
NaiveBayesTextClassifierTrainer_out.result

<p style = 'font-size:16px;font-family:Arial'>Now let us predict from our test table.</p> 

In [ ]:
help(NaiveBayesTextClassifierPredict)

In [ ]:
complaints_tokens_test = DataFrame("complaints_tokens_test")
complaints_tokens_test

In [ ]:
nbt_predict_out = NaiveBayesTextClassifierPredict(newdata=complaints_tokens_test,
                                                  object=NaiveBayesTextClassifierTrainer_out,
                                                  input_token_column='token',
                                                  doc_id_columns='doc_id',
                                                  model_token_column='token',
                                                  model_category_column='category',
                                                  model_prob_column='prob',
                                                  newdata_partition_column='doc_id')

In [ ]:
nbt_predict_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
db_drop_table("complaints")

In [ ]:
db_drop_table("stop_words")

In [ ]:
db_drop_table("complaints_tokens_test")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>NaiveBayesTextClassifierTrainer function reference: <a href = 'https://docs.teradata.com/search/all?query=NaiveBayesTextClassifierTrainer&content-lang=en-US'>here</a></li>
        <li>NaiveBayesTextClassifierPredict function reference: <a href = 'https://docs.teradata.com/search/all?query=NaiveBayesTextClassifierPredict&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>